In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
service = Service()

# if you want to get fancy with the options
# here the the ChromeOptions documentation: https://chromedriver.chromium.org/capabilities#h.p_ID_102
# here is the list of Chrome arguments: https://peter.sh/experiments/chromium-command-line-switches/
# there are a lot...
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True) # this keeps the window open post execution/errors/ changing frames (explained later)

prefs = {"download.default_directory": '/Users/justinpaul/Downloads',  # note need to change this
         "directory_upgrade": True}
chrome_options.add_experimental_option("prefs", prefs)

In [46]:
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://us.shein.com/')
WebDriverWait(driver, timeout=10)

#click out of coupon pop up
x_box = driver.find_element(By.CLASS_NAME, "icon-close").click()

input = 'shirt'

search_bar = driver.find_element(By.CLASS_NAME, "header-search-input")
search_bar.send_keys(input)

search_enter = driver.find_element(By.CLASS_NAME, "suiiconfont-critical").click()

parent_size=driver.find_element(By.CSS_SELECTOR, "[aria-label=Size]")
view_more=parent_size.find_element(By.CLASS_NAME, "side-filter__item-viewMore").click()

size = "M"
size_boxes = parent_size.find_elements(By.CLASS_NAME, "sui-checkbox__label-text")
for size_box in size_boxes:
  if size_box.text == size:
    size_box.click()


In [52]:
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

items = soup.find_all('section', class_="product-list__item")

list_of_lists = []

for item in items:
  list_of_item = []
  #item_a = item.find('div', class_="S-product-item__name").find('a')
  item_a = item.find('a')
  href_value = item_a.get('href')

  link = 'us.shein.com' + href_value
  list_of_item.append(link)

  item_name = item_a.text
  list_of_item.append(item_name)

  item_price = item.find('p', class_="product-item__camecase-wrap").find('span').text
  list_of_item.append(item_price)

  list_of_lists.append(list_of_item)

df = pd.DataFrame(list_of_lists, columns=['link', 'item name', 'price'])

csv_tracker = open('csv_tracker.txt', 'r')
csv_number = csv_tracker.read()
print(csv_number)
df.to_csv(f'{csv_number}.csv', index=False, sep=',')

#print('csv_tracker_write')
csv_tracker_write = open("csv_tracker.txt", "w")
next_csv_number = str(int(csv_number) + 1)
print(next_csv_number)
csv_tracker_write.write(next_csv_number)

2
3


1

In [53]:
print(csv_number)
prev_csv_num = int(csv_number) - 1
print(prev_csv_num)
df_prev = pd.read_csv(f'{prev_csv_num}.csv')
df = pd.read_csv(f'{csv_number}.csv')

#print(df.head())
#print(df_prev.head())

#for i in range(len(df.index)):
#print(df.columns)
#for index, row in df.iterrows():
#  print(row['item_name'])

df_concat = pd.concat([df, df_prev], keys=['x', 'y'])
#print(df_concat.to_string())

duplicate_links = df_concat['link'].duplicated()
#print(duplicate_links)
#print(len(duplicate_links))

link = df_concat['link'][0]

print(df)
price_current = float(df.loc[df['link'] == link]['price'].to_string(index=False).strip("$"))
print(price_current)
price_prev = float(df_prev.loc[df['link'] == link]['price'].to_string(index=False).strip("$"))
print(price_prev)


price_drops = []
for i in range(len(duplicate_links)):
  if duplicate_links[i] == True:
    link = df_concat['link'][i]

    price_current = float(df.loc[df['link'] == link]['price'].to_string(index=False).strip("$"))
    price_prev = float(df_prev.loc[df['link'] == link]['price'].to_string(index=False).strip("$"))

    price_drop = price_prev - price_current

    item_with_price_drop = [link, price_drop]

    price_drops.append(item_with_price_drop)


df_price_drops = pd.DataFrame(price_drops, columns=['link', 'price_drop'])

print(df_price_drops.nlargest(n=5, columns='price_drop'))
#print(df_price_drops.sort_values(by=['price_drop']).head(5))


2
1
                                                  link     item name  price
0    us.shein.com/SHEIN-LUNE-Feather-Print-Keyhole-...  17SAVE $0.67  $3.82
1    us.shein.com/Manfinity-Mode-Men-Horse-Print-Co...  10SAVE $5.50  $6.99
2    us.shein.com/SHEIN-LUNE-Plaid-Print-Button-Fro...            11  $7.53
3    us.shein.com/SHEIN-LUNE-Solid-Cold-Shoulder-Sp...  16SAVE $1.20  $6.29
4    us.shein.com/Manfinity-Homme-Men-Ribbed-Knit-P...    SAVE $3.38  $9.61
..                                                 ...           ...    ...
115  us.shein.com/SHEIN-LUNE-Ditsy-Floral-Flounce-S...             8  $8.49
116  us.shein.com/Manfinity-Homme-Men-Argyle-Jacqua...   6SAVE $2.62  $9.87
117  us.shein.com/SHEIN-Priv-Polka-Dot-Ruffle-Trim-...    SAVE $2.30  $9.19
118  us.shein.com/SHEIN-Clasi-Gold-Dot-Print-Flare-...   4Save $1.50  $5.99
119  us.shein.com/Manfinity-Homme-Men-Drop-Shoulder...   8SAVE $4.75  $8.74

[120 rows x 3 columns]
3.82
3.82
                                                li

/var/folders/02/9y8vd0t555l0mp4010rn9w140000gn/T/ipykernel_3876/2223976143.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link = df_concat['link'][0]
/var/folders/02/9y8vd0t555l0mp4010rn9w140000gn/T/ipykernel_3876/2223976143.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if duplicate_links[i] == True:
/var/folders/02/9y8vd0t555l0mp4010rn9w140000gn/T/ipykernel_3876/2223976143.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[p